In [3]:
# Select the TensorFlow 2.0 runtime
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [4]:
# Install Weights and Biases (WnB)
#!pip install wandb

In [5]:
# Primary imports
import tensorflow as tf
import numpy as np
import wandb

In [8]:
# Load the FashionMNIST dataset, scale the pixel values
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [9]:
# Define the labels of the dataset
CLASSES=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [10]:
# Change the pixel values to float32 and reshape input data
X_train = X_train.astype("float32").reshape(-1, 28, 28, 1)
X_test = X_test.astype("float32").reshape(-1, 28, 28, 1)

In [11]:
y_train.shape, y_test.shape

((60000,), (10000,))

In [12]:
# TensorFlow imports
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [13]:
# Define utility function for building a basic shallow Convnet 
def get_training_model():
    model = Sequential()
    model.add(Conv2D(16, (5, 5), activation="relu",
        input_shape=(28, 28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dense(len(CLASSES), activation="softmax"))
    
    return model

In [14]:
# Define loass function and optimizer
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
#optimizer = tf.keras.optimizers.Adam()
optimizer_adam = tf.keras.optimizers.Adam()
optimizer_adadelta = tf.keras.optimizers.Adadelta()

In [15]:
train_loss_adam = tf.keras.metrics.Mean(name="train_loss")
valid_loss_adam = tf.keras.metrics.Mean(name="test_loss")

# Specify the performance metric
train_acc_adam= tf.keras.metrics.SparseCategoricalAccuracy(name="train_acc")
valid_acc_adam = tf.keras.metrics.SparseCategoricalAccuracy(name="valid_acc")

In [16]:
# Batches of 64
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)

In [17]:
# Train the model
@tf.function
def model_train_adam(features, labels):
    # Define the GradientTape context
    with tf.GradientTape() as tape:
        # Get the probabilities
        predictions = model(features)
        # Calculate the loss
        loss = loss_func(labels, predictions)
    # Get the gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    # Update the weights
    #optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    optimizer_adam.apply_gradients(zip(gradients, model.trainable_variables))

    # Update the loss and accuracy
    train_loss_adam(loss)
    train_acc_adam(labels, predictions)

In [18]:
# Validating the model
# Validating the model
@tf.function
def model_validate(features, labels):
    predictions = model(features)
    v_loss = loss_func(labels, predictions)

    valid_loss(v_loss)
    valid_acc(labels, predictions)
@tf.function
def model_validate_adam(features, labels):
    predictions = model(features)
    v_loss = loss_func(labels, predictions)

    valid_acc_adam(v_loss)
    valid_acc_adam(labels, predictions)
    
@tf.function
def model_validate_adadelta(features, labels):
    predictions = model(features)
    v_loss = loss_func(labels, predictions)

    valid_acc_adadelta(v_loss)
    valid_acc_adadelta(labels, predictions)

In [19]:
# A shallow Convnet
model = get_training_model()

In [20]:
# Grab random images from the test and make predictions using 
# the model *while it is training* and log them using WnB
def get_sample_predictions():
    predictions = []
    images = []
    random_indices = np.random.choice(X_test.shape[0], 25)
    for index in random_indices:
        image = X_test[index].reshape(1, 28, 28, 1)
        prediction = np.argmax(model(image).numpy(), axis=1)
        prediction = CLASSES[int(prediction)]
        
        images.append(image)
        predictions.append(prediction)
    
    wandb.log({"predictions": [wandb.Image(image, caption=prediction) 
                               for (image, prediction) in zip(images, predictions)]})

In [22]:
# Train the model for 5 epochs
for epoch in range(10):
    # Run the model through train and test sets respectively
    for (features, labels) in train_ds:
        model_train_adam(features, labels)
        model_train_adadelta(features, labels)

    for test_features, test_labels in test_ds:
        model_validate_adam(test_features, test_labels)
        model_validate_adadelta(test_features, test_labels)
        
    # Grab the results
    (loss_adadelta, acc_adadelta) = train_loss_adadelta.result(), train_acc_adadelta.result()
    (val_loss_adadelta, val_acc_adadelta) = valid_loss_adadelta.result(), valid_acc_adadelta.result()
    
    # Clear the current state of the metrics
    train_loss_adadelta.reset_states(), train_acc_adadelta.reset_states()
    valid_loss_adadelta.reset_states(), valid_acc_adadelta.reset_states()
    
    # Local logging
    template = "Epoch {}, loss: {:.3f}, acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}"
    print (template.format(epoch+1,
                         loss_adadelta,
                         acc_adadelta,
                         val_loss_adadelta,
                         val_acc_adadelta))
    
    # Logging with WnB
    wandb.log({"train_loss_adadelta": loss_adadelta.numpy(),
               "train_accuracy_adadelta": acc_adadelta.numpy(),
               "val_loss_adadelta": val_loss_adadelta.numpy(),
               "val_accuracy_adadelta": val_acc_adadelta.numpy()
    })
    # adam
    # Grab the results
    (loss_adam, acc_adam) = train_loss_adam.result(), train_acc_adam.result()
    (val_loss_adam, val_acc_adam) = valid_loss_adam.result(), valid_acc_adam.result()
    
    # Clear the current state of the metrics
    train_loss_adam.reset_states(), train_acc_adam.reset_states()
    valid_loss_adam.reset_states(), valid_acc_adam.reset_states()
    
    # Local logging
    template = "Epoch {}, loss: {:.3f}, acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}"
    print (template.format(epoch+1,
                         loss_adam,
                         acc_adam,
                         val_loss_adam,
                         val_acc_adam))
    
    # Logging with WnB
    wandb.log({"train_loss_adadelta": loss_adam.numpy(),
               "train_accuracy_adadelta": acc_adam.numpy(),
               "val_loss_adadelta": val_loss_adam.numpy(),
               "val_accuracy_adadelta": val_acc_adam.numpy()
    })
    get_sample_predictions()

TypeError: in converted code:

    <ipython-input-18-bad58be2c3cc>:7 model_validate_adam  *
        valid_acc_adam(v_loss)
    /Users/rdua/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/metrics.py:193 __call__
        replica_local_fn, *args, **kwargs)
    /Users/rdua/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/distribute/distributed_training_utils.py:1135 call_replica_local_fn
        return fn(*args, **kwargs)
    /Users/rdua/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/metrics.py:176 replica_local_fn
        update_op = self.update_state(*args, **kwargs)  # pylint: disable=not-callable
    /Users/rdua/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/metrics_utils.py:75 decorated
        update_op = update_state_fn(*args, **kwargs)

    TypeError: update_state() missing 1 required positional argument: 'y_pred'
